# Procesado de votos con join en Spark

Este notebook es una alternativa a `process-votes-with-KSQL`. En este caso leeremos la tabla de municipios directamente en Spark, en vez de depender del preprocesado con Connect y KSQL. Este notebook simplemente incluye el join y la consulta del dashboard principal. El otro notebook incluye consultas intermedias para practicar con el procesado de streams en spark.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import from_json
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

PACKAGES = "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0,org.xerial:sqlite-jdbc:3.27.2"
spark = SparkSession \
    .builder \
    .appName("StructuredVotesSparkOnly") \
    .config("spark.jars.packages", PACKAGES)\
    .getOrCreate()



In [2]:
from ejercicios.votes import TOPIC_VOTES

La lectura desde un SQL en Spark es tan sencilla como:

In [3]:
properties = {'driver': 'org.sqlite.JDBC', 'date_string_format': 'yyyy-MM-dd HH:mm:ss'}
municipios = spark.read.jdbc("jdbc:sqlite:/tmp/municipios.db", "municipios", properties=properties) \
    .withColumnRenamed('Codigo', 'CODIGO') \
    .withColumnRenamed('Comunidad', 'COMUNIDAD') \
    .withColumnRenamed('Provincia', 'PROVINCIA') \
    .withColumnRenamed('Municipio', 'MUNICIPIO') \

In [4]:
municipios.show(5, False)

+------+---------+---------+---------+---------+
|CODIGO|COMUNIDAD|PROVINCIA|MUNICIPIO|Poblacion|
+------+---------+---------+---------+---------+
|4001  |Andalucía|Almería  |Abla     |1342     |
|4002  |Andalucía|Almería  |Abrucena |1279     |
|4003  |Andalucía|Almería  |Adra     |24670    |
|4004  |Andalucía|Almería  |Albánchez|805      |
|4005  |Andalucía|Almería  |Alboloduy|653      |
+------+---------+---------+---------+---------+
only showing top 5 rows



In [18]:
municipios.printSchema()

root
 |-- CODIGO: integer (nullable = true)
 |-- COMUNIDAD: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- Poblacion: integer (nullable = true)



Ahora leemos el topic de VOTES, en vez de VOTES_ENRICHED.

In [5]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", TOPIC_VOTES) \
  .load()


Los mensajes llegan en formato JSON, pero al contrario que con `spark.read.csv`, debemos indicar el esquema completo (OJO, este esquema sólo tiene CODIGO y PARTIDO porque el topic VOTES aún no se ha combinado con la tabla estática).

In [6]:
schema = StructType([
    StructField("Codigo", IntegerType()),
    StructField("Partido", StringType())
])

La función de verificación de firma es igual en ambos notebooks.

In [7]:
def process_signature(comunidad, provincia, municipio):
    return 'OK'

udf_process_signature = fn.udf(process_signature)

Finalmente construimos la query de procesado de votos. Si la comparamos con la query del primer notebook, ésta sólo incluye las transformaciones `join` y `withColumnRenamed` (para evita cambiar el resto del código).

In [25]:
query = df \
    .selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING) AS value") \
    .withColumn("value_json", fn.from_json(col('value'), schema)) \
    .select('value_json.Codigo', 'value_json.Partido') \
    .withColumnRenamed('Codigo', 'CODIGO') \
    .withColumnRenamed('Partido', 'PARTIDO') \
    .join(municipios, 'CODIGO', 'inner') \
    .withColumn('SIGNATURE', udf_process_signature(col('COMUNIDAD'), col('PROVINCIA'), col('MUNICIPIO'))) \
    .where(~ fn.isnull(col('SIGNATURE'))) \
    .groupBy('COMUNIDAD', 'PROVINCIA', 'PARTIDO') \
    .agg(fn.count('*').alias('VOTOS')) \
    .sort(col('COMUNIDAD').asc(), col('PROVINCIA').asc(), col('VOTOS').desc()) \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName('dashboard') \
    .start()


Y leemos el dashboard igual que en el otro notebook.

In [29]:
spark.sql("""
SELECT COMUNIDAD, PARTIDO, sum(VOTOS) as VOTOS
  FROM dashboard
  WHERE VOTOS > 2 and COMUNIDAD LIKE 'And%'
  GROUP BY COMUNIDAD, PARTIDO
  ORDER BY VOTOS DESC
""").show(100, False)

+---------+-------------------------+-----+
|COMUNIDAD|PARTIDO                  |VOTOS|
+---------+-------------------------+-----+
|Andalucía|Cs                       |83   |
|Andalucía|PSOE                     |77   |
|Andalucía|Nulos                    |71   |
|Andalucía|PODEMOS_IU_EQUO          |68   |
|Andalucía|Blanco                   |61   |
|Andalucía|PP                       |59   |
|Andalucía|PACMA                    |58   |
|Andalucía|RECORTES_CERO_GRUPO_VERDE|48   |
|Andalucía|PCPE                     |47   |
|Andalucía|UPyD                     |45   |
|Andalucía|VOX                      |40   |
|Andalucía|FE_de_las_JONS           |18   |
|Andalucía|IZAR                     |14   |
|Andalucía|AND                      |10   |
|Andalucía|PCOE                     |9    |
|Andalucía|EB                       |9    |
|Andalucía|CILUS                    |4    |
+---------+-------------------------+-----+



In [17]:
query.stop()